In [31]:
import ebooklib
from ebooklib import epub
import re
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from scipy.sparse import random
from sklearn.decomposition import TruncatedSVD
from nltk.corpus import words
import gensim
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import words, stopwords, names
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse import random
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import simple_preprocess
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
import gensim
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import nltk
from bs4 import BeautifulSoup
import ebooklib
from ebooklib import epub
import os
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from sklearn.neighbors import NearestNeighbors


In [7]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\76392\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\76392\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
def read_epub_book(epub_file, ID):
    book = epub.read_epub(epub_file)
    text = ""

    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            content = item.get_content().decode('utf-8')
            content = re.sub('<[^<]+?>', '', content)  # Remove HTML tags
            content = re.sub('\s+', ' ', content)  # Replace multiple whitespaces with a single space
            content = re.sub('\n', ' ', content)
            content = re.sub("&#13;", ' ', content)
            text += content
            #paragraphs.extend(content.strip().split("&#13;"))
    book_item = {'text': text, 'bookID':ID}
    return book_item

def merge_strings_until_limit(strings, min_length, max_length, test_for_max = 0):
    merged_string = ""
    merged_strings = []

    for s in strings:
        if len(merged_string) <= min_length:# and len(s)<max_length:
            merged_string += s
        
        elif len(merged_string) > max_length and test_for_max<5:
            splitParagraph = merged_string.split('.')
            splitParagraphRePoint = []
            for sp in splitParagraph:
                splitParagraphRePoint.append(sp+'.')
            merged = merge_strings_until_limit(splitParagraphRePoint, min_length, max_length, test_for_max+1)
            merged_strings.extend(merged)
            merged_string = s
        else:
            merged_strings.append(merged_string)
            merged_string = s

    if merged_string:
        merged_strings.append(merged_string)

    return merged_strings

def read_epub_paragraphs(epub_file, ID):
    book = epub.read_epub(epub_file)
    paragraphs = []

    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            content = item.get_content().decode('utf-8')
            content = re.sub('<[^<]+?>', '', content)  # Remove HTML tags
            content = re.sub('\s+', ' ', content)  # Replace multiple whitespaces with a single space
            content = re.sub('\n', ' ', content)
            paragraphs.extend(content.strip().split("&#13;"))
    paragraphs = merge_strings_until_limit(paragraphs, 200, 1000)
    paragraphs = [{'paragraph':paragraphs[i], 'nr':i,'bookID':ID} for i in range(len(paragraphs)) if len(paragraphs[i])>50]
    return paragraphs[1:-1]

In [3]:
def extract_text_from_pdf(pdf_path):
    text = {}
    pdf_document = fitz.open(pdf_path)
    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)
        textInfo = page.get_text("blocks", flags=1+2+8)
        paragraphs = [re.sub('\n', ' ', t[4]) for t in textInfo]
        text[page_num] = paragraphs
    pdf_document.close()
    return text

def extract_images_from_pdf(pdf_path, image_output_dir):
    pdf_document = fitz.open(pdf_path)
    images = []

    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)
        image_list = page.get_images(full=True)

        for img_index, img_info in enumerate(image_list):
            image = pdf_document.extract_image(img_info[0])
            image_path = f"{image_output_dir}/page_{page_num + 1}_image_{img_index}.png"
            with open(image_path, "wb") as image_file:
                image_file.write(image["image"])

            images.append(image_path)

    pdf_document.close()
    return images

In [23]:
def preprocess(paragraphs):
    STOP_WORDS = set(stopwords.words("english"))
    LEMMATIZER = WordNetLemmatizer()
    STEMMER = PorterStemmer()
    
    processed_docs = [] 
    
    for paragraph in paragraphs:
        words = gensim.utils.simple_preprocess(paragraph, min_len=3, deacc=True)        
        lemmatized_words = [LEMMATIZER.lemmatize(word) for word in words]
        
        filtered_words = [word for word in lemmatized_words if (word not in STOP_WORDS) and (is_english_word(word))]
        
        stemmed_words = [STEMMER.stem(word) for word in filtered_words]
        
        processed_doc = " ".join(stemmed_words)
        processed_docs.append(processed_doc)  

    return processed_docs

def preprocess2(paragraphs):
    lemmatizer = WordNetLemmatizer()
    STOP_WORDS = set(stopwords.words("english"))

    processed_docs = []

    for paragraph in paragraphs:
        words = gensim.utils.simple_preprocess(paragraph, min_len=3, deacc=True)
        lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
        filtered_words = [word for word in lemmatized_words if word not in STOP_WORDS]
        processed_docs.append(filtered_words)
    
    return processed_docs

In [14]:
def read_multiple_epub(epub_files):
    IDs = [os.path.splitext(os.path.basename(e))[0] for e in epub_files]
    all_paragraphs = []
    for epub_file, ID in zip(epub_files, IDs):
        paragraphs = read_epub_paragraphs(epub_file, ID)
        all_paragraphs.extend(paragraphs)
    return all_paragraphs

In [21]:
english_words = set(words.words())

def is_english_word(word):
    return word in english_words


In [11]:
import glob

In [16]:
BOOK_DIR = r'D:\YeqinHUANG_Collection\ucl\week14_28.Jan.2024\06text\epub'
BOOK_FILES = []
BOOK_FILES.extend(glob.glob(os.path.join(BOOK_DIR, '*.epub')))

In [17]:
PARAGRAPHS = read_multiple_epub(BOOK_FILES)

C:\Users\76392\anaconda3\Lib\site-packages\ebooklib\epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')


In [66]:
import pickle
with open('Paragraph.obj','wb') as pickleFile:
    pickle.dump(PARAGRAPHS, pickleFile)

In [68]:
with open('Paragraph.obj','rb') as pickleFile:
    PARAGRAPHS_LOADED = pickle.load(pickleFile)

In [24]:
PREPROCESSED = preprocess([p['paragraph'] for p in PARAGRAPHS])

In [26]:
PREPROCESSED2 = preprocess([p['paragraph'] for p in PARAGRAPHS])

# TF-IDF

In [27]:
VECTORIZER_TFIDF = TfidfVectorizer(min_df=2)
TFIDF_MATRIX = VECTORIZER_TFIDF.fit_transform(PREPROCESSED)

In [33]:
def search_tfidf(query, k=1):
    model = NearestNeighbors(n_neighbors = k,
                            metric = 'cosine',
                            algorithm = 'brute',
                            n_jobs = -1)
    model.fit(TFIDF_MATRIX)
    qv = get_tfidf_vector(query)
    neighbours = model.kneighbors(qv, k, return_distance = False)[0]
    paragraphs = [PARAGRAPHS[i] for i in neighbours]
    return paragraphs

def get_tfidf_vector(query):
    query_vector = VECTORIZER_TFIDF.transform([query])
    return query_vector

In [35]:
search_tfidf('hello',5)

[{'paragraph': ' “Hello, Jake. Hello!” Mike called. “Come here. I want you to meet my friends. We’re all having an hors–d’œuvre.” I was introduced to the people at the table. They supplied their names to Mike and sent for a fork for me.',
  'nr': 927,
  'bookID': 'hemingway-sun-also-rises'},
 {'paragraph': ' I’ve been wanting to talk to you.” “Hello, Frances,” said Cohn. He smiled. “Why, hello, Robert. Are you here?” She went on, talking rapidly. “I’ve had the darndest time. This one”—shaking her head at Cohn—“didn’t come home for lunch.',
  'nr': 258,
  'bookID': 'hemingway-sun-also-rises'},
 {'paragraph': ' “Here come the gentry,” Bill said. They were crossing the street “Hello, men,” I said. “Hello, gents!” said Brett. “You saved us seats? How nice.” “I say,” Mike said, “that Romero what’shisname is somebody.',
  'nr': 978,
  'bookID': 'hemingway-sun-also-rises'},
 {'paragraph': ' “Have another port?” “All right,” said Harvey. “There comes Cohn,” I said. Robert Cohn was crossing the

# SVD

In [37]:
n_components = 250
SVD = TruncatedSVD(n_components = n_components, algorithm = 'randomized')
REDUCED_MATRIX = SVD.fit_transform(TFIDF_MATRIX)

In [38]:
def get_svd_vector(query):
    tfidf_v = get_tfidf_vector(query)
    svd_v = SVD.transform(tfidf_v)
    return svd_v

In [41]:
def search_svd(query, k=1):
    model = NearestNeighbors(n_neighbors = k,
                            metric='cosine',
                            algorithm='brute',
                            n_jobs=-1)
    model.fit(REDUCED_MATRIX)    
    qv = get_svd_vector(query)
    neighbours = model.kneighbors(qv, k, return_distance=False)[0]
    paragraphs = [PARAGRAPHS[i] for i in neighbours]
    return paragraphs

In [44]:
def search_svd_k(query, k=1):
    model = NearestNeighbors(n_neighbors = k,
                            metric = 'cosine',
                            algorithm = 'brute',
                            n_jobs = -1)
    model.fit(REDUCED_MATRIX)    
    qv = get_svd_vector(query)
    neighbours = model.kneighbors(qv, k, return_distance = False)[0]
    paragraphs = [PARAGRAPHS[i] for i in neighbours]
    return paragraphs    

In [45]:
query = 'hello'

In [46]:
search_svd_k(query)

[{'paragraph': ' “Hello, Jake. Hello!” Mike called. “Come here. I want you to meet my friends. We’re all having an hors–d’œuvre.” I was introduced to the people at the table. They supplied their names to Mike and sent for a fork for me.',
  'nr': 927,
  'bookID': 'hemingway-sun-also-rises'}]

# Doc2Vec

In [48]:
from gensim.models.doc2vec import TaggedDocument, Doc2Vec

DOC2VEC_DOCUMENTS = [TaggedDocument(doc, [i]) for i, doc in enumerate(PREPROCESSED2)]

DOC2VEC_MODEL = Doc2Vec(vector_size=250, min_count=0, alpha=0.025, min_alpha=0.025, epochs=100)
DOC2VEC_MODEL.build_vocab(DOC2VEC_DOCUMENTS)
DOC2VEC_MODEL.train(DOC2VEC_DOCUMENTS, total_examples=DOC2VEC_MODEL.corpus_count, epochs=100)

In [49]:
def get_doc2vec_vector(query):
    tokens = preprocess2([query])[0]
    vector = DOC2VEC_MODEL.infer_vector(tokens)
    return vector

In [53]:
def search_doc2vec(query, k=1):
    vectorQuery = get_doc2vec_vector(query)
    neighbors = DOC2VEC_MODEL.dv.most_similar(positive=[vectorQuery], topn=k)
    paragraphs = [PARAGRAPHS[n[0]] for n in neighbors]
    return paragraphs

In [56]:
search_doc2vec(query,5)

[{'paragraph': ' But this treaty was purchased at an expense which might have supported a vigorous and successful war; and the subjects of Theodosius were compelled to redeem the safety of a worthless favorite by oppressive taxes, which they would more cheerfully have paid for his destruction.',
  'nr': 1930,
  'bookID': 'gibbon-history-of-the-decline-and-fall-of-the-roman-empire-volume-3'},
 {'paragraph': ' The impatient crowd rushed at the dawn of day to secure their places, and there were many who passed a sleepless and anxious night in the adjacent porticos. From the morning to the evening, careless of the sun, or of the rain, the spectators, who sometimes amounted to the number of four hundred thousand, remained in eager attention; their eyes fixed on the horses and charioteers, their minds agitated with hope and fear, for the success of the colors which they espoused: and the happiness of Rome appeared to hang on the event of a race.',
  'nr': 1082,
  'bookID': 'gibbon-history-of

# 4

In [60]:
def searchAllThree(query):
    tfidf = search_tfidf(query)
    svd = search_svd(query)
    doc2vec = search_doc2vec(query)
    
    print('TFIDF:{0}\n SVD: {1}\n doc2vec:{2}'.format(tfidf, svd, doc2vec))

In [61]:
query = 'running around university'

In [62]:
searchAllThree(query)

TFIDF:[{'paragraph': '” “You haven’t been around much, have you?” “Yes, my dear. I have been around very much. I have been around a very great deal.” “Drink your wine,” said Brett. “We’ve all been around. I dare say Jake here has seen as much as you have.', 'nr': 351, 'bookID': 'hemingway-sun-also-rises'}]
 SVD: [{'paragraph': '” “You haven’t been around much, have you?” “Yes, my dear. I have been around very much. I have been around a very great deal.” “Drink your wine,” said Brett. “We’ve all been around. I dare say Jake here has seen as much as you have.', 'nr': 351, 'bookID': 'hemingway-sun-also-rises'}]
 doc2vec:[{'paragraph': ' The sectaries were gradually disqualified from the possession of honorable or lucrative employments; and Theodosius was satisfied with his own justice, when he decreed, that, as the Eunomians distinguished the nature of the Son from that of the Father, they should be incapable of making their wills or of receiving any advantage from testamentary donations.